In [1]:
from classes import *
from helper_functions import *
import pandas as pd
import numpy as np

import datetime
from datetime import date

from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from tqdm import tqdm

In [3]:
# Importing the dataset and configuration parameters
df = pd.read_csv('data/test01.csv')

factory_coord = os.getenv("FACTORY_GEO_COORD") 

In [4]:
# Convert datetime columns to their correct types respectively
df['est_installation_date'] = pd.to_datetime(df['est_installation_date'],dayfirst=True)

#Sort according to the [deadline date, distance to customer] for optimally processing the jobs
df = df.sort_values(by=['est_installation_date','travel_time'], ascending=[True,True],ignore_index=True)

In [5]:
# Planning dates from next_working_date
first_date = next_working_date(date.today())

In [6]:
df.head(10)

,id,job_geo_coordinate,start_geo_coordinate,travel_time,installers_required,expected_job_time,est_installation_date,installation_date,arrival_start_time,arrival_end_time,INSTALLER_IDs
0,70307,"-34.7996109,138.7078888","-34.7810071,138.6461490",13.0,1,120.0,2023-01-14,NaN,NaN,NaN,NaN
1,70920,"-34.8307492,138.7029293","-34.7810071,138.6461490",14.0,1,60.0,2023-01-14,NaN,NaN,NaN,NaN
2,71451,"-34.7705843,138.7344186","-34.7810071,138.6461490",15.0,1,60.0,2023-01-14,NaN,NaN,NaN,NaN
3,71097,"-34.8727383,138.6442177","-34.7810071,138.6461490",19.0,1,90.0,2023-01-14,NaN,NaN,NaN,NaN
4,71069,"-34.878409,138.5853056","-34.7810071,138.6461490",19.0,1,240.0,2023-01-14,NaN,NaN,NaN,NaN
5,70825,"-34.8847309,138.6721202","-34.7810071,138.6461490",21.0,1,120.0,2023-01-14,NaN,NaN,NaN,NaN
6,71374,"-34.8815709,138.6713833","-34.7810071,138.6461490",21.0,1,120.0,2023-01-14,NaN,NaN,NaN,NaN
7,70822,"-34.8922422,138.613558","-34.7810071,138.6461490",22.0,2,240.0,2023-01-14,NaN,NaN,NaN,NaN
8,69844,"-34.887275,138.6506171","-34.7810071,138.6461490",23.0,1,90.0,2023-01-14,NaN,NaN,NaN,NaN
9,70566,"-34.9009458,138.6297647","-34.7810071,138.6461490",23.0,1,60.0,2023-01-14,NaN,NaN,NaN,NaN


In [14]:
# Creating test_installers
num_installers = 5
installers = []
for i in range(num_installers):
    installer = Installer(i)
    
    #Initializing last_job_end times as 8 am for each installer
    installer.end_time = datetime.datetime.combine(first_date,datetime.time()) + datetime.timedelta(hours=8) ## Time at which they leave factory
    installers.append(installer)

In [15]:
for index, row in tqdm(df.iterrows()):
    try:
        # Check first_date < deadline
        if first_date > row["est_installation_date"].date():
            print(index,'Unable to process dates before ' + first_date)

        #Check if enough installers available
        if row["installers_required"] > num_installers:
            print(index,'Not enough installers available for this job')

        # Calculate which installers can reach next job location earliest
        ins_start_time_nextJob = []
        ins_travel_times = []
        for installer in installers:
            time_to_travel = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            ins_travel_times.append(time_to_travel)
            if installer.time_spent + time_to_travel + row["expected_job_time"] > 480:                       # Can add drive back time here
                # Shift to next working day
                next_date = next_working_date(installer.end_time.date())
                installer.start_time = None
                installer.end_time = datetime.datetime.combine(next_date,datetime.time()) + datetime.timedelta(hours=8)
                installer.reset_location()
                installer.reset_time_spent()

            arrival_time = installer.end_time + datetime.timedelta(minutes=time_to_travel)                   # arrival time for next job
            ins_start_time_nextJob.append((installer.id,arrival_time.strftime('%Y-%m-%d %H:%M:%S')))
        ins_start_time_nextJob = sorted(ins_start_time_nextJob,key=lambda x: x[1])

        # Installers to send
        ins_to_send = []
        for i in range(row["installers_required"]):
            ins_to_send.append(ins_start_time_nextJob[i][0])

        # Update installers' variables
        curr_max = datetime.datetime.combine(date.today(),datetime.time()) # Start time for the installer who arrives last
        for ins_id in ins_to_send:
            installer = installers[ins_id]
            curr_travel_time = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            prev_job_end_time = installer.end_time
            installer.start_time = prev_job_end_time + datetime.timedelta(minutes=curr_travel_time)
            curr_max = max(curr_max,installer.start_time)

        for ins_id in ins_to_send:
            installer = installers[ins_id]
            curr_travel_time = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            prev_job_end_time = installer.end_time
            installer.avail = False
            installer.start_time = curr_max
            installer.end_time = installer.start_time + datetime.timedelta(minutes=row["expected_job_time"])
            installer.time_spent += row["expected_job_time"] + curr_travel_time
            installer.geo_coord = row["job_geo_coordinate"]

        job_start_time = curr_max
        job_end_time = curr_max + datetime.timedelta(minutes=row["expected_job_time"])
        installer_ids = ins_to_send

        df.loc[index,'installation_date'] = job_start_time.date()
        df.loc[index,'arrival_start_time'] = job_start_time.time()
        df.loc[index,'arrival_end_time'] = job_end_time.time()
        df.loc[index,'INSTALLER_IDs'] = ','.join(map(str,installer_ids))

        # # print(job_start_time,'\t',job_end_time,'\t',installer_ids)
        # if df.loc[index,'installation_date'] > row["est_installation_date"]:
        #     print(index,'Cant accomodate for', row["est_installation_date"])
    except:
        # print(index)
        continue

477it [01:35,  5.00it/s]


In [54]:
df.to_csv('ram_ram.csv',index=False)